In [ ]:
import pandas
import numpy as np
import requests

SANITY_PER_DAY = 240 + 60 # natural + monthly
SANITY_RESERVE = 0 # Sum of potions saved up
SANITY_PER_OP = 135 # depends on your level

Load data, code taken from [PeterYR](https://colab.research.google.com/drive/12RYH2DYD_c-H7uZJ_gkOzpCFcVFVOB70#scrollTo=Se9SZJSLD3eM)

In [ ]:
response = requests.get('https://penguin-stats.io/PenguinStats/api/v2/result/matrix?show_closed_zones=true')
df = pandas.DataFrame.from_records(response.json()['matrix'])

response = requests.get('https://raw.githubusercontent.com/Kengxxiao/ArknightsGameData/master/zh_CN/gamedata/excel/stage_table.json')
stagedict = response.json()

response = requests.get('https://raw.githubusercontent.com/Kengxxiao/ArknightsGameData/master/zh_CN/gamedata/excel/building_data.json')
craftdict = response.json()

## en_US names
response = requests.get('https://raw.githubusercontent.com/Kengxxiao/ArknightsGameData/master/en_US/gamedata/excel/item_table.json')
itemdict = response.json()

## Include reruns
rerun_suffix = "_rep"
suffix_len = len(rerun_suffix)
known_stage_ids = np.array(np.unique(df.stageId), dtype="U32")
rerun_indices = np.flatnonzero(np.core.defchararray.find(known_stage_ids,rerun_suffix)!=-1)
rerun_stage_ids = known_stage_ids[rerun_indices]
rerun_original_stages = np.empty(len(rerun_stage_ids), dtype="U32")

## Distinguish tough stages
tough_prefix = "tough_"
prefix_len = len(tough_prefix)
known_stage_ids = np.array(np.unique(df.stageId), dtype="U32")
tough_indices = np.flatnonzero(np.core.defchararray.find(known_stage_ids,tough_prefix)!=-1)
tough_stage_ids = known_stage_ids[tough_indices]

for i, stage_id in enumerate(rerun_stage_ids):
    rerun_original_stages[i] = stage_id[:-suffix_len]

SAN_COST_DICT = {} # stageId -> sanity cost
STAGE_NAME_DICT = {} # stageName -> stageId
REVERSE_STAGE_NAME_DICT = {} # stageId -> stageName

for stageId, info in stagedict['stages'].items():
    ## Reject challenge mode stages
    if stageId[-1] == '#':
        continue
        
    apCost = info['apCost']
    code = info['code']
    print(info.keys())
    break
    
    if stageId in tough_stage_ids:
        code += " (tough)"
    
    SAN_COST_DICT[stageId] = apCost
    STAGE_NAME_DICT[code] = stageId
    REVERSE_STAGE_NAME_DICT[stageId] = code
    
    potential_rerun_id = stageId+rerun_suffix
    if potential_rerun_id in rerun_stage_ids:
        code += " Rerun"
        SAN_COST_DICT[potential_rerun_id] = apCost
        STAGE_NAME_DICT[code] = potential_rerun_id
        REVERSE_STAGE_NAME_DICT[potential_rerun_id] = code

    
ITEM_NAME_DICT = {} # itemName -> itemId
REVERSE_ITEM_NAME_DICT = {} # itemId -> itemName

for itemId, info in itemdict['items'].items():
    itemName = info['name']
    ITEM_NAME_DICT[itemName] = itemId
    REVERSE_ITEM_NAME_DICT[itemId] = itemName
    
    

In [ ]:
def get_drop_percent(stageName: str, itemName: str) -> float:
    stageId = STAGE_NAME_DICT[stageName]
    itemId  = ITEM_NAME_DICT[itemName]
    #print(stageId, itemId)
    
    #stage_df = df
    stage_df = df[df.stageId == stageId]
    item_df = stage_df[stage_df.itemId == itemId]
    #print(item_df)
    
    percent = float(item_df.quantity/item_df.times)
    
    return percent

In [ ]:
def get_san_per_drop(stageName: str, itemName: bool) -> float:
    percent = get_drop_percent(stageName, itemName)
    stage_sanity = SAN_COST_DICT[STAGE_NAME_DICT[stageName]]
    return stage_sanity/percent

In [ ]:
def get_expected_sanity(stageName: str, itemName: str, nDrops: int) -> int:
    #percent = get_drop_percent(stageName, itemName)
    expected_sanity_per_item = get_san_per_drop(stageName, itemName)
    expected_total_sanity = nDrops * expected_sanity_per_item
    return expected_total_sanity

Which stage and item do you want to farm?

In [ ]:
stageName = "NL-10"
itemName = "Loxic Kohl"
nDrops =  300
eventLength = 10 # days

In [ ]:
expected_total_sanity = get_expected_sanity(stageName, itemName, nDrops)
print("Expected sanity: {}".format(int(np.ceil(expected_total_sanity))))

sanity_to_spend = eventLength * SANITY_PER_DAY + SANITY_RESERVE
leftover_sanity = expected_total_sanity - sanity_to_spend

needed_op = leftover_sanity / SANITY_PER_OP
needed_op = int(np.ceil(needed_op))

print("Needed OP: {}".format(needed_op))

In [ ]:
n_op_per_day = 4
daily_spend = SANITY_PER_DAY + n_op_per_day*SANITY_PER_OP
expected_farming_length = expected_total_sanity/daily_spend

print("Will take {:.1f} days at {:d} OP per day".format(expected_farming_length, n_op_per_day))

In [ ]:
wanted_farming = [
    ["NL-10", "Loxic Kohl", 300],
    ["NL-9", "Crystalline Component", 150],
]
eventLength = 14 # days
sanity_reserve = 0

total_san = 0
for s, i, n in wanted_farming:
    total_san += get_expected_sanity(s, i, n)
    
print("Expected sanity: {}".format(int(np.ceil(total_san))))

sanity_to_spend = eventLength * SANITY_PER_DAY + sanity_reserve
leftover_sanity = total_san - sanity_to_spend

needed_op = leftover_sanity / SANITY_PER_OP
needed_op = int(np.ceil(needed_op))

print("Needed OP: {}".format(needed_op))

In [ ]:
def get_best_stage_for_item(itemName: str) -> str:
    itemId  = ITEM_NAME_DICT[itemName]
    item_df = df[df.itemId == itemId]
    
    exp_san = np.zeros(len(item_df))
    
    for i, d in enumerate(item_df.values):
        try:
            sid = d[0]
            cost = SAN_COST_DICT[sid]
            rate = d[3]/d[2]
            if d[2] < 1000:
                raise Exception("E")
            exp_san[i] = rate * cost
        except Exception as e:
            #print(e)
            pass
    
    #exp_san = np.trim_zeros(exp_san)
    sorted_indices = np.flip(np.argsort(exp_san))
    
    for i in sorted_indices:
        try:
            sid = item_df.values[i][0]
            print(REVERSE_STAGE_NAME_DICT[sid])
        except Exception as e:
            #print(sid)
            pass
    #print(item_df["stageId"][sorted_indices])
    
get_best_stage_for_item("Oriron Cluster")

## Calcing MSV

https://www.reddit.com/r/arknights/comments/ggdjiu/on_the_calculation_of_material_sanity_value_and/

In [ ]:

stage_ids = np.array(np.unique(df.stageId), dtype="U32")
item_ids  = np.array(np.unique(df.itemId), dtype="U32")

#san_keys = [k for k in SAN_COST_DICT.keys()]
#stage_ids = np.intersect1d(stage_ids, san_keys)
main_prefix = "main"
main_indices = np.flatnonzero(np.core.defchararray.find(stage_ids,main_prefix)!=-1)
#stage_ids = stage_ids[main_indices]

sub_indices = np.flatnonzero(np.core.defchararray.find(stage_ids,"sub")!=-1)
keep_indices = np.concatenate((main_indices,sub_indices))
stage_ids = stage_ids[keep_indices]


named_items = np.array([k for k in REVERSE_ITEM_NAME_DICT.keys()])
item_names = [k for k in REVERSE_ITEM_NAME_DICT.values()]
chip_substr = " Chip"
non_chip_idx = np.flatnonzero(np.core.defchararray.find(item_names,chip_substr)==-1)
named_items = named_items[non_chip_idx]
item_ids = np.intersect1d(item_ids, named_items)[:47]

extras = [
    "D32 Steel",
    "Bipolar Nanoflake",
    "Polymerization Preparation",
    "Crystalline Circuit",
    "Pure Gold",
    "Drill Battle Record",
    "Frontline Battle Record",
    "Tactical Battle Record",
    "Strategic Battle Record",
]
extras_ids = []
for e in extras:
    extras_ids.append(ITEM_NAME_DICT[e])
item_ids = np.concatenate((item_ids, extras_ids))

n_stages = len(stage_ids)
n_items = len(item_ids)

print("N items: {}\nN stages: {}".format(n_items, n_stages))

stage_san = np.zeros(n_stages)
item_msv = np.zeros(n_items)

for i, s in enumerate(stage_ids):
    try:
        stage_san[i] = SAN_COST_DICT[s]
        #print(s)
    except Exception as e:
        #print("Error {}".format(e))
        pass

#for i in item_ids:
#    print(REVERSE_ITEM_NAME_DICT[i])


In [ ]:
print(item_ids)

In [ ]:
drop_matrix = np.zeros((n_stages, n_items))

for v in df.values:
    try:
        stage_idx = np.where(stage_ids == v[0])[0][0]
        item_idx = np.where(item_ids == v[1])[0][0]
        drop_matrix[stage_idx, item_idx] = v[3]/v[2]
    except:
        pass

stages_with_drops = np.sum(drop_matrix, axis=1)
no_drops = np.where(stages_with_drops > 0)
drop_matrix = drop_matrix[no_drops]
stage_san = stage_san[no_drops]

no_san = np.where(stage_san > 0)[0]
drop_matrix = drop_matrix[no_san]
stage_san = stage_san[no_san]

n_stages = len(stage_san)

#np.savetxt("./test.txt", drop_matrix)

In [ ]:
#initials = np.genfromtxt("./msv_inputs.txt", delimiter=",", usecols=1)
#print(len(initials))

In [ ]:
#for i, v in enumerate(item_ids):
#    try:
#        print(REVERSE_ITEM_NAME_DICT[v])
#    except Exception as e:
#        print("Error: {} Missing {}".format(i, e))

In [ ]:
## solve problem:
## drop_matrix * item_msv - stage_san = stage_ret
## constraints:
##   maximise sum(stage_ret)
##   each stage_ret <= 0
## 
## i.e.
## drop_matrix * item_msv <= stage_san
## constraints:
##   minimise -sum(drop_matrix * item_msv)
##   drop_matrix * item_msv <= stage_san

from scipy.optimize import NonlinearConstraint, Bounds, LinearConstraint
from scipy.optimize import minimize, linprog

In [ ]:
craft_matrix = np.zeros((n_items, n_items))
subprod_matrix = np.zeros((n_items, n_items))
lmd_cost = np.zeros(n_items)

recipes = [v for v in craftdict["workshopFormulas"].values()]

## Assume only one recipe per item
for r in recipes:
    item_idx = np.where(item_ids == r["itemId"])[0]

    if len(item_idx) <= 0:
        continue
        
    lmd_cost[item_idx] = r["goldCost"]
    
    for c in r["costs"]:
        c_idx = np.where(item_ids == c["id"])[0]
        if len(c_idx) <= 0:
            continue
        #print(np.shape(craft_matrix[item_idx]))
        #print(item_idx)
        craft_matrix[item_idx,c_idx] = float(c["count"])
    
    total_w = 0
    for s in r["extraOutcomeGroup"]:
        total_w += float(s["weight"])
        s_idx = np.where(item_ids == s["itemId"])[0]
        subprod_matrix[item_idx,s_idx] = float(s["weight"])
        
    subprod_matrix[item_idx] /= total_w

In [ ]:
byprod_rate = 0.18
craft_constraint_matrix = np.identity(n_items) - craft_matrix + byprod_rate * subprod_matrix

taken_lmd_value = 0.004
lmd_san_hack = lmd_cost * taken_lmd_value
#lmd_san_hack = np.ones(len(lmd_cost))

has_lmd = np.where(lmd_cost > 0)[0]
lmd_san_hack = lmd_san_hack[has_lmd]
craft_constraint_matrix = craft_constraint_matrix[has_lmd]

In [ ]:

stage_ret_lambda = lambda x: np.matmul(drop_matrix, x)
total_ret_lambda = lambda x: -np.sum(stage_ret_lambda(x))

C1 = LinearConstraint(drop_matrix, lb=0, ub=stage_san)

coeff = np.sum(drop_matrix, axis=0)
res = linprog(-coeff,
              A_ub=drop_matrix, b_ub=stage_san,
              A_eq=craft_constraint_matrix, b_eq=-lmd_san_hack,
              bounds=(0, None))
print(res)

In [ ]:
## Make dict from itemId -> msv at the end
msv = res.x
msv_dict = {}
for i, d in enumerate(item_ids):
    msv_dict[d] = msv[i]
    print("{}: {:.3f}".format(REVERSE_ITEM_NAME_DICT[d], msv[i]))

In [ ]:
def get_drop_rate(stage_name: str, item_name: str) -> float:
    stage_id = STAGE_NAME_DICT[stage_name]
    item_id = ITEM_NAME_DICT[item_name]
    stage_idx = np.where(stage_ids == stage_id)[0][0]
    item_idx = np.where(item_ids == item_id)[0][0]
    
    return drop_matrix[stage_idx][item_idx]

print(get_drop_rate("1-1", "Orirock"))

In [ ]:
#print(stage_san)
idx = np.where(item_ids == ITEM_NAME_DICT["Sugar Pack"])[0]
idxm = np.where(item_ids == ITEM_NAME_DICT["Incandescent Alloy Block"])[0]
print(100*subprod_matrix[idxm,idx])
print(craft_matrix[idxm,idx])

In [ ]:
craft_row = craft_matrix[idxm]
required = np.where(craft_row > 0)[1]
for i in required:
    print(REVERSE_ITEM_NAME_DICT[item_ids[i]])

In [ ]:
#print(ITEM_NAME_DICT["D32 Steel"])

In [ ]:
zero_lmd = np.where(lmd_cost <= 0)[0]
print(subprod_matrix[zero_lmd])
print(lmd_cost)

In [ ]:
extras = [
    "D32 Steel",
    "Bipolar Nanoflake",
    "Polymerization Preparation",
    "Crystalline Circuit"
]
for v in extras:
    print(ITEM_NAME_DICT[v])

In [ ]:
san_profits = np.matmul(drop_matrix,msv) - stage_san
for i, s in enumerate(stage_ids):
    
    print("{}: {:.2f}".format(REVERSE_STAGE_NAME_DICT[s], san_profits[i]))

In [ ]:
print(ITEM_NAME_DICT.keys())

## Using ak_requests

In [1]:
import ak_requests as ak
import numpy as np
from scipy.optimize import linprog

%load_ext autoreload
%autoreload 2

In [4]:
item_dict = ak.get_item_dict(lang="en_US")
stage_dict = ak.get_stage_dict()
craft_dict = ak.get_craft_dict()
psdf = ak.get_pengstats_df(server="CN")

In [5]:
stage_san_cost, stage_names, stage_names_rev, stage_lmd_dict = ak.get_stage_info(psdf, stage_dict)
item_names, item_names_rev = ak.get_item_info(item_dict)
event_names, event_names_rev = ak.get_event_info(stage_names)

In [6]:
material_ids = ak.get_material_ids(item_names_rev, ak.MATERIAL_NAMES)
n_mats_all = len(material_ids)

In [7]:
#main_stage_ids = ak.get_main_stage_ids(psdf)
main_stage_ids = ak.get_main_and_perm_stage_ids(psdf)
stage_ids = np.concatenate((main_stage_ids,[
    "wk_melee_5",
    "wk_kc_5",
    "wk_fly_5"
]))

stage_san_all = ak.get_san_cost(stage_ids, stage_san_cost)
n_stages_all = len(stage_ids)

In [8]:
craft_matrix, subprod_matrix = ak.get_craft_matrix(craft_dict, material_ids)

In [9]:
drop_matrix_all, coeff_matrix_all = ak.get_drop_matrix(psdf, stage_ids, material_ids, stage_lmd_dict)

Error: ['act16d5_09_rep' 'furni' 3520 113 1641801600000 1642622400000.0] "'act16d5_09_rep'"
Error: ['act16d5_09_rep' '30033' 3520 2915 1641801600000 1642622400000.0] "'act16d5_09_rep'"
Error: ['act16d5_03_rep' 'furni' 45 1 1641801600000 1642622400000.0] "'act16d5_03_rep'"
Error: ['act16d5_03_rep' '30041' 45 81 1641801600000 1642622400000.0] "'act16d5_03_rep'"
Error: ['act16d5_03_rep' '30011' 45 79 1641801600000 1642622400000.0] "'act16d5_03_rep'"
Error: ['act13d5_02_perm' '30031' 7 0 1634241600000 nan] "'act13d5_02_perm'"
Error: ['act13d5_02_perm' '30011' 7 0 1634241600000 nan] "'act13d5_02_perm'"
Error: ['act13d5_02_perm' '30051' 7 0 1634241600000 nan] "'act13d5_02_perm'"
Error: ['act13d5_02_perm' '30012' 7 9 1634241600000 nan] "'act13d5_02_perm'"
Error: ['act13d5_02_perm' 'furni' 7 0 1634241600000 nan] "'act13d5_02_perm'"
Error: ['act13d5_02_perm' '2001' 7 7 1634241600000 nan] "'act13d5_02_perm'"
Error: ['act13d5_02_perm' '30021' 7 0 1634241600000 nan] "'act13d5_02_perm'"
Error: ['ac

Error: ['act5d0_08_rep' '2002' 2 10 1606809600000 1607630400000.0] "'act5d0_08_rep'"
Error: ['act5d0_08_rep' 'furni' 2 0 1606809600000 1607630400000.0] "'act5d0_08_rep'"
Error: ['act18d0_02_perm' '30051' 3 2 1633032000000 nan] "'act18d0_02_perm'"
Error: ['act18d0_02_perm' '30052' 3 0 1633032000000 nan] "'act18d0_02_perm'"
Error: ['act18d0_02_perm' '2001' 3 0 1633032000000 nan] "'act18d0_02_perm'"
Error: ['act18d0_02_perm' '30022' 3 0 1633032000000 nan] "'act18d0_02_perm'"
Error: ['act18d0_02_perm' '2002' 3 1 1633032000000 nan] "'act18d0_02_perm'"
Error: ['act18d0_02_perm' '30042' 3 0 1633032000000 nan] "'act18d0_02_perm'"
Error: ['act18d0_02_perm' 'furni' 3 0 1633032000000 nan] "'act18d0_02_perm'"
Error: ['act18d0_02_perm' '30021' 3 0 1633032000000 nan] "'act18d0_02_perm'"
Error: ['act18d0_02_perm' '30041' 3 2 1633032000000 nan] "'act18d0_02_perm'"
Error: ['act15d0_06_rep' 'furni' 3311 91 1638864000000 1639684800000.0] "'act15d0_06_rep'"
Error: ['act15d0_06_rep' '30023' 3311 2269 16388

In [10]:
drop_matrix, stage_san = ak.filter_drop_matrix(drop_matrix_all, stage_san_all)

In [11]:
byprod_rate = 0.18
craft_constraint_matrix = ak.get_craft_constraint_matrix(craft_matrix, subprod_matrix, byprod_rate)

In [12]:
coeff = np.sum(drop_matrix, axis=0)
res = linprog(-coeff,
              A_ub=drop_matrix, b_ub=stage_san,
              A_eq=craft_constraint_matrix, b_eq=np.zeros(len(craft_constraint_matrix)),
              bounds=(0, None)
             )

print(res.success)
## Make dict from itemId -> msv at the end
msv = res.x
msv_dict = {}
for i, d in enumerate(material_ids):
    msv_dict[d] = msv[i]
    print("{}: {:.3f}".format(item_names[d], msv[i]))

True
LMD: 0.004
Pure Gold: 9.499
Drill Battle Record: 0.772
Frontline Battle Record: 1.544
Tactical Battle Record: 3.859
Strategic Battle Record: 7.718
Skill Summary - 1: 1.565
Skill Summary - 2: 3.979
Skill Summary - 3: 10.115
Orirock: 0.868
Orirock Cube: 2.679
Orirock Cluster: 13.211
Orirock Concentration: 49.640
Damaged Device: 3.069
Device: 9.282
Integrated Device: 36.943
Optimized Device: 90.328
Ester: 1.795
Polyester: 5.462
Polyester Pack: 21.660
Polyester Lump: 91.937
Sugar Substitute: 1.820
Sugar: 5.535
Sugar Pack: 21.951
Sugar Lump: 93.331
Oriron Shard: 2.131
Oriron: 6.468
Oriron Cluster: 25.684
Oriron Block: 106.770
Diketon: 2.279
Polyketon: 6.913
Aketon: 27.465
Keton Colloid: 100.625
Loxic Kohl: 24.353
White Horse Kohl: 64.572
Manganese Ore: 26.945
Manganese Trihydrate: 96.702
Grindstone: 30.166
Grindstone Pentahydrate: 89.592
RMA70-12: 21.470
RMA70-24: 72.154
Polymerization Preparation: 243.818
Bipolar Nanoflake: 206.256
D32 Steel: 245.231
Coagulating Gel: 28.289
Polymerize

In [13]:
ak.print_craft_materials("Cutting Fluid Solution", craft_matrix, material_ids, item_names, item_names_rev)

LMD: 300
RMA70-12: 1
Crystalline Component: 1
Compound Cutting Fluid: 1


In [27]:
ak.print_stage_drops("R8-11", drop_matrix, stage_ids, material_ids, stage_names_rev, item_names)

LMD: 252
Crystalline Component: 52.49%
Orirock Cube: 36.86%
Orirock: 21.31%
Device: 13.91%
Pure Gold: 10.23%
Damaged Device: 7.80%
Orirock Cluster: 3.26%
Loxic Kohl: 2.68%
Crystalline Circuit: 2.66%
Coagulating Gel: 2.12%
Integrated Device: 1.84%


In [15]:
print(stage_names_rev["LS-5"])

wk_kc_5


In [16]:
print(item_names_rev["Pure Gold"])

3003


In [26]:
event_ids = ak.get_event_ids("NL", psdf, event_names_rev)
event_stage_san = ak.get_san_cost(event_ids, stage_san_cost)
event_drop_matrix, _ = ak.get_drop_matrix(psdf, event_ids, material_ids, stage_lmd_dict)
#sanity_return = np.matmul(event_drop_matrix, msv)
print(ak.get_stage_efficiency(event_drop_matrix, msv, event_stage_san))

[0.55210271 0.54157622 0.51224042 0.73320216 0.86629975 0.7191438
 0.86698108 1.04683048 0.95159901 1.10618295]
